<img src = "https://escp.eu/sites/default/files/logo/ESCP-logo-white-misalign.svg" width = 400 style="background-color: #240085;">
<h1 align=center><font size = 6>ESCP Business School</font></h1>
<h3 align=center><font size = 5>ESCP Hackathon</font><br/>
<font size = 3>Project 3: Brand sentiment analysis | Criteo</font></h3>
<h6 align=center>Chapter 0: Data Mining - Twitter</h6>

Last Updated: March 16, 2021\
Author: Group 07 - Arya Bhattacharya, Hanadi Yachoui, Ming Hin Chung, Resha Dirga

<h3>Chapter Objectives</h3>

<p>This chapter aims to send GET request to social media we would like to monitor. On this chapter we will send GET request to <b>Twitter</b> and process them so that the data are ready to use on the next steps.</p>

In [1]:
# Import dependencies
import requests
import json

import numpy as np
import pandas as pd

import tweepy
import time

In [2]:
# Setup authorisation keys
API_KEY = # Insert your user API Key here
API_KEY_SECRET = # Insert your user API Key Secret here
BEARER_TOKEN = # Insert your user Bearer Token here
ACCESS_TOKEN = # Insert your user Access Token here
ACCESS_TOKEN_SECRET = # Insert your user Access Token Secret here
label = # Insert your environment label here

In [4]:
# Create function to generate search tweet command to Twitter API
def form_search_url(search_key, start_date, end_date, lang, next_):
    print('Creating search url for', search_key)
    url_strings = []

    url_strings.append("https://api.twitter.com/1.1/tweets/search/30day/"+label+".json?query=(")
    url_strings.append(search_key)
    if len(lang):
        url_strings.append("%20lang%3A"+lang[0])

    url_strings.append(")&fromDate="+start_date)
    url_strings.append("&toDate="+end_date)
    url_strings.append("&maxResults=100")
    
    if next_ != 'empty':
        url_strings.append("&next="+next_)

    url_=''.join(url_strings)
    time.sleep(2)
    return url_


In [5]:
# Create function to generate search user command to Twitter API
def form_user_url(id_):
    print('Creating search url for')
    url_strings = []

    url_strings.append("https://api.twitter.com/2/users/")
    url_strings.append(str(id_))
    url_strings.append("?user.fields=id,created_at,name,username,protected,verified,withheld,location,description,public_metrics")

    url_=''.join(url_strings)
    time.sleep(2)
    return url_


In [6]:
# Create function to send GET request for tweet search to Twitter API
def get_tweets(url_):
    print('Getting tweets')
    url     = url_
    headers = {'Authorization': 'Bearer '+BEARER_TOKEN}
    res = requests.get(url, headers=headers)
    json_data = json.loads(res.text)
    
    try:
        next_ = json_data['next']
    except:
        next_ = 'empty'
    try:
        return {'results': json_data['results'], 'next': next_}
    except:
        return print(json_data)

In [19]:
# Create function to send GET request for user search to Twitter API
def get_users(url_):
    print('Retrieving users')
    url     = url_
    headers = {'Authorization': 'Bearer '+BEARER_TOKEN}
    res = requests.get(url, headers=headers)
    data = res.json()
    try:
        return data['data']
    except:
        return data

In [8]:
# Create function to translate obtained tweets to dataframe
def translate_tweet(data):
    print('Translating tweet data')
    tweet = data

    post_ids = []
    post_times = []
    reply_to = []
    author_ids = []
    texts = []
    mentions = []
    sources = []
    langs = []
    user_ids = []

    # Get Tweet ID    
    try:
        post_ids.append(tweet['id'])
    except:
        post_ids.append(np.nan)

    # Get post time
    try:
        post_times.append(tweet['created_at'])
    except:
        post_times.append(np.nan)

    # Get reply to (if any)
    try:
        reply_to.append(tweet['in_reply_to_user_id'])
    except:
        reply_to.append(np.nan)

    # Get author ID
    try:
        author_ids.append(tweet['user']['id'])
        user_ids.append(tweet['user']['id'])
    except:
        author_ids.append(np.nan)

    # Get tweet text
    try:
        texts.append(tweet['text'])
    except:
        texts.append(np.nan)

    # Get mentions
    try:
        mentions_ = tweet['entities']['user_mentions']
        temp = []
        for mention in mentions_:
            temp.append(mention['screen_name'])
            user_ids.append(mention['id'])
        mentions.append(temp)
    except:
        mentions.append(np.nan)

    # Get device type
    try:
        sources.append(tweet['source'].split('>')[1].split('<')[0])
    except:
        sources.append(np.nan)

    # Get language
    try:
        langs.append(tweet['lang'])
    except:
        langs.append(np.nan)

    result = {
        'id': post_ids,
        'created_at': post_times,
        'in_reply_to_user_id': reply_to,
        'author_id': author_ids,
        'text': texts,
        'mentions': mentions,
        'source': sources,
        'lang': langs
    }

    df_tweets = pd.DataFrame(result)
    return {'tweets': df_tweets, 'users': user_ids}

In [21]:
# Create function to send GET request for user lists to Twitter API
def get_lists(id_):
    url = "https://api.twitter.com/2/users/"+str(id_)+"/list_memberships?max_results=10&list.fields=created_at,description,follower_count,id,member_count,name&expansions=owner_id"
    headers = {'Authorization': 'Bearer '+BEARER_TOKEN}
    time.sleep(2)
    res = requests.get(url, headers=headers)
    data = res.json()
    
    try:
        lists = []
        for list_ in data['data']:
            lists.append(list_['name'])
        return lists
    except:
        return np.nan


In [10]:
# Create function to translate obtained users to dataframe
def translate_users(data):
    print('Translating user data')

    user = data

    user_ids = []
    user_names = []
    user_usernames = []
    user_locations = []
    user_followers = []
    user_following = []
    user_tweets = []
    user_listed_counts = []
    user_verifieds = []
    user_created_ats = []

    # Get User ID    
    try:
        user_ids.append(user['id'])
    except:
        user_ids.append(np.nan)

    # Get User names
    try:
        user_names.append(user['name'])
    except:
        user_names.append(np.nan)

    # Get User username
    try:
        user_usernames.append(user['username'])
    except:
        user_usernames.append(np.nan)

    # Get User location
    try:
        user_locations.append(user['location'])
    except:
        user_locations.append(np.nan)

    # Get # of followers
    try:
        user_followers.append(user['public_metrics']['followers_count'])
    except:
        user_followers.append(np.nan)

    # Get # of followers
    try:
        user_following.append(user['public_metrics']['following_count'])
    except:
        user_following.append(np.nan)

    # Get # of tweets
    try:
        user_tweets.append(user['public_metrics']['tweet_count'])
    except:
        user_tweets.append(np.nan)

    # Get listed count
    try:
        user_listed_counts.append(user['public_metrics']['listed_count'])
    except:
        user_listed_counts.append(np.nan)

    # Get verified status
    try:
        user_verifieds.append(user['verified'])
    except:
        user_verifieds.append(np.nan)

    # Get account creation date
    try:
        user_created_ats.append(user['created_at'])
    except:
        user_created_ats.append(np.nan)

    result = {
        'id': user_ids,
        'name': user_names,
        'username': user_usernames,
        'location': user_locations,
        'followers_count': user_followers,
        'following_count': user_following,
        'tweet_count': user_tweets,
        'listed_count': user_listed_counts,
        'verified': user_verifieds,
        'created_at': user_created_ats
    }

    df_users = pd.DataFrame(result)
    df_users['lists'] = df_users.apply(lambda x: get_lists(x.id), axis=1)

    return df_users


<hr/>
<h5>Get tweets</h5>

In [11]:
# Assign search parameters
search_key = "Decathlon"
lang = ["en"]
count_limit = 10
start_date = '202203040000'
end_date = '202203170000'
count = 0

# Perform request
list_ids = []

df_tweet = pd.DataFrame()
df_user = pd.DataFrame()

url_ = form_search_url(search_key, start_date, end_date, lang, 'empty')

tweets = get_tweets(url_)
tweets_result = tweets['results']
next_ = tweets['next']

for tweet in tweets_result:
    tweet_ = translate_tweet(tweet)
    df_tweet_temp = pd.DataFrame(tweet_['tweets'])
    list_ids.extend(tweet_['users'])
    df_tweet = pd.concat([df_tweet, df_tweet_temp], axis=0)

while next_ != 'empty':
    url_ = form_search_url(search_key, start_date, end_date, lang, next_)

    tweets = get_tweets(url_)
    tweets_result = tweets['results']
    next_ = tweets['next']

    for tweet in tweets_result:
        tweet_ = translate_tweet(tweet)
        df_tweet_temp = pd.DataFrame(tweet_['tweets'])
        list_ids.extend(tweet_['users'])
        df_tweet = pd.concat([df_tweet, df_tweet_temp], axis=0)
        
print('Job complete!')

Creating search url for Decathlon
Getting tweets
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translat

Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Creating search url for Decathlon
Getting tweets
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translat

Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Creating search url for Decathlon
Getting tweets
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translat

Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Creating search url for Decathlon
Getting tweets
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translat

Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Creating search url for Decathlon
Getting tweets
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translat

Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Creating search url for Decathlon
Getting tweets
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translat

Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Creating search url for Decathlon
Getting tweets
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translat

Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Creating search url for Decathlon
Getting tweets
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translat

Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Creating search url for Decathlon
Getting tweets
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translat

Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Translating tweet data
Job complete!


In [52]:
# Create function to clean tweets from retweets
def clean_tweets(text):
    clean_text = text.split('RT ')
    if ~len(clean_text[0]):
        try:
            return clean_text[1]
        except:
            return clean_text[0]
    else:
        return clean_text[0]

In [53]:
# Clean tweets from retweets
df_tweet['text_clean'] = df_tweet.apply(lambda x: clean_tweets(x.text), axis=1)
df_tweet

,id,created_at,in_reply_to_user_id,author_id,text,mentions,source,lang,text_clean
0,1503882954375929866,Tue Mar 15 23:56:51 +0000 2022,None,612834201,RT @WrocWW: @AnonOpsSE I found few words to #L...,"[WrocWW, AnonOpsSE]",Twitter Web App,en,@WrocWW: @AnonOpsSE I found few words to #Lero...
0,1503882686380879877,Tue Mar 15 23:55:47 +0000 2022,None,612834201,"RT @Ro_manoff: @tashecon Also, please pay atte...","[Ro_manoff, tashecon]",Twitter Web App,en,"@Ro_manoff: @tashecon Also, please pay attenti..."
0,1503881899328032771,Tue Mar 15 23:52:40 +0000 2022,None,4174651769,RT @NoButYeahBut_: #BoycottDecathlon!!!\n#Deca...,[NoButYeahBut_],Twitter for Android,en,@NoButYeahBut_: #BoycottDecathlon!!!\n#Decathl...
0,1503880374874607620,Tue Mar 15 23:46:36 +0000 2022,126267113,51585491,@Decathlon @DecathlonUK why are you still doin...,"[Decathlon, DecathlonUK]",Twitter for iPhone,en,@Decathlon @DecathlonUK why are you still doin...
0,1503880275318775809,Tue Mar 15 23:46:13 +0000 2022,None,4174651769,"RT @TrudeauMaggie: Auchan, Leroy Merlin, Decat...",[TrudeauMaggie],Twitter for Android,en,"@TrudeauMaggie: Auchan, Leroy Merlin, Decathlo..."
...,...,...,...,...,...,...,...,...,...
0,1499543192596725760,Fri Mar 04 00:32:11 +0000 2022,None,360737613,RT @BoosterVista: Kudos to the MVHS Academic D...,[BoosterVista],Twitter for iPhone,en,@BoosterVista: Kudos to the MVHS Academic Deca...
0,1499541366673747971,Fri Mar 04 00:24:56 +0000 2022,None,3316370737,"RT @jwilliams_mhs: As a team, MHS Academic Dec...",[jwilliams_mhs],Twitter for iPhone,en,"@jwilliams_mhs: As a team, MHS Academic Decath..."
0,1499541283886485507,Fri Mar 04 00:24:36 +0000 2022,None,768215814520524801,"As a team, MHS Academic Decathlon came away fr...",[],Twitter for iPhone,en,"As a team, MHS Academic Decathlon came away fr..."
0,1499538730570465284,Fri Mar 04 00:14:28 +0000 2022,None,1488628891916849156,33239. \nTriathlon n. Athletic contest of thr...,[],botjeelaa,en,33239. \nTriathlon n. Athletic contest of thr...


In [54]:
# Export tweet dataframe
df_tweet.to_csv('tweets_'+start_date+'.csv')

<hr/>
<h5>Get users detail</h5>

In [22]:
# Perform request to get users details
list_ids = list(set(list_ids))

for id_ in list_ids:
    print(str(list_ids.index(id_)+1) + '/' + str(len(list_ids)))
    url_ = form_user_url(id_)
    user_result = get_users(url_)
    result = translate_users(user_result)
    df_user = pd.concat([df_user, result], axis=0)

1/3623
Creating search url for
Retrieving users
Translating user data
2/3623
Creating search url for
Retrieving users
Translating user data
3/3623
Creating search url for
Retrieving users
Translating user data
4/3623
Creating search url for
Retrieving users
Translating user data
5/3623
Creating search url for
Retrieving users
Translating user data
6/3623
Creating search url for
Retrieving users
Translating user data
7/3623
Creating search url for
Retrieving users
Translating user data
8/3623
Creating search url for
Retrieving users
Translating user data
9/3623
Creating search url for
Retrieving users
Translating user data
10/3623
Creating search url for
Retrieving users
Translating user data
11/3623
Creating search url for
Retrieving users
Translating user data
12/3623
Creating search url for
Retrieving users
Translating user data
13/3623
Creating search url for
Retrieving users
Translating user data
14/3623
Creating search url for
Retrieving users
Translating user data
15/3623
Creatin

Retrieving users
Translating user data
117/3623
Creating search url for
Retrieving users
Translating user data
118/3623
Creating search url for
Retrieving users
Translating user data
119/3623
Creating search url for
Retrieving users
Translating user data
120/3623
Creating search url for
Retrieving users
Translating user data
121/3623
Creating search url for
Retrieving users
Translating user data
122/3623
Creating search url for
Retrieving users
Translating user data
123/3623
Creating search url for
Retrieving users
Translating user data
124/3623
Creating search url for
Retrieving users
Translating user data
125/3623
Creating search url for
Retrieving users
Translating user data
126/3623
Creating search url for
Retrieving users
Translating user data
127/3623
Creating search url for
Retrieving users
Translating user data
128/3623
Creating search url for
Retrieving users
Translating user data
129/3623
Creating search url for
Retrieving users
Translating user data
130/3623
Creating search 

Retrieving users
Translating user data
231/3623
Creating search url for
Retrieving users
Translating user data
232/3623
Creating search url for
Retrieving users
Translating user data
233/3623
Creating search url for
Retrieving users
Translating user data
234/3623
Creating search url for
Retrieving users
Translating user data
235/3623
Creating search url for
Retrieving users
Translating user data
236/3623
Creating search url for
Retrieving users
Translating user data
237/3623
Creating search url for
Retrieving users
Translating user data
238/3623
Creating search url for
Retrieving users
Translating user data
239/3623
Creating search url for
Retrieving users
Translating user data
240/3623
Creating search url for
Retrieving users
Translating user data
241/3623
Creating search url for
Retrieving users
Translating user data
242/3623
Creating search url for
Retrieving users
Translating user data
243/3623
Creating search url for
Retrieving users
Translating user data
244/3623
Creating search 

Retrieving users
Translating user data
345/3623
Creating search url for
Retrieving users
Translating user data
346/3623
Creating search url for
Retrieving users
Translating user data
347/3623
Creating search url for
Retrieving users
Translating user data
348/3623
Creating search url for
Retrieving users
Translating user data
349/3623
Creating search url for
Retrieving users
Translating user data
350/3623
Creating search url for
Retrieving users
Translating user data
351/3623
Creating search url for
Retrieving users
Translating user data
352/3623
Creating search url for
Retrieving users
Translating user data
353/3623
Creating search url for
Retrieving users
Translating user data
354/3623
Creating search url for
Retrieving users
Translating user data
355/3623
Creating search url for
Retrieving users
Translating user data
356/3623
Creating search url for
Retrieving users
Translating user data
357/3623
Creating search url for
Retrieving users
Translating user data
358/3623
Creating search 

Retrieving users
Translating user data
459/3623
Creating search url for
Retrieving users
Translating user data
460/3623
Creating search url for
Retrieving users
Translating user data
461/3623
Creating search url for
Retrieving users
Translating user data
462/3623
Creating search url for
Retrieving users
Translating user data
463/3623
Creating search url for
Retrieving users
Translating user data
464/3623
Creating search url for
Retrieving users
Translating user data
465/3623
Creating search url for
Retrieving users
Translating user data
466/3623
Creating search url for
Retrieving users
Translating user data
467/3623
Creating search url for
Retrieving users
Translating user data
468/3623
Creating search url for
Retrieving users
Translating user data
469/3623
Creating search url for
Retrieving users
Translating user data
470/3623
Creating search url for
Retrieving users
Translating user data
471/3623
Creating search url for
Retrieving users
Translating user data
472/3623
Creating search 

...

Retrieving users
Translating user data
3163/3623
Creating search url for
Retrieving users
Translating user data
3164/3623
Creating search url for
Retrieving users
Translating user data
3165/3623
Creating search url for
Retrieving users
Translating user data
3166/3623
Creating search url for
Retrieving users
Translating user data
3167/3623
Creating search url for
Retrieving users
Translating user data
3168/3623
Creating search url for
Retrieving users
Translating user data
3169/3623
Creating search url for
Retrieving users
Translating user data
3170/3623
Creating search url for
Retrieving users
Translating user data
3171/3623
Creating search url for
Retrieving users
Translating user data
3172/3623
Creating search url for
Retrieving users
Translating user data
3173/3623
Creating search url for
Retrieving users
Translating user data
3174/3623
Creating search url for
Retrieving users
Translating user data
3175/3623
Creating search url for
Retrieving users
Translating user data
3176/3623
Cr

Translating user data
3275/3623
Creating search url for
Retrieving users
Translating user data
3276/3623
Creating search url for
Retrieving users
Translating user data
3277/3623
Creating search url for
Retrieving users
Translating user data
3278/3623
Creating search url for
Retrieving users
Translating user data
3279/3623
Creating search url for
Retrieving users
Translating user data
3280/3623
Creating search url for
Retrieving users
Translating user data
3281/3623
Creating search url for
Retrieving users
Translating user data
3282/3623
Creating search url for
Retrieving users
Translating user data
3283/3623
Creating search url for
Retrieving users
Translating user data
3284/3623
Creating search url for
Retrieving users
Translating user data
3285/3623
Creating search url for
Retrieving users
Translating user data
3286/3623
Creating search url for
Retrieving users
Translating user data
3287/3623
Creating search url for
Retrieving users
Translating user data
3288/3623
Creating search url

3387/3623
Creating search url for
Retrieving users
Translating user data
3388/3623
Creating search url for
Retrieving users
Translating user data
3389/3623
Creating search url for
Retrieving users
Translating user data
3390/3623
Creating search url for
Retrieving users
Translating user data
3391/3623
Creating search url for
Retrieving users
Translating user data
3392/3623
Creating search url for
Retrieving users
Translating user data
3393/3623
Creating search url for
Retrieving users
Translating user data
3394/3623
Creating search url for
Retrieving users
Translating user data
3395/3623
Creating search url for
Retrieving users
Translating user data
3396/3623
Creating search url for
Retrieving users
Translating user data
3397/3623
Creating search url for
Retrieving users
Translating user data
3398/3623
Creating search url for
Retrieving users
Translating user data
3399/3623
Creating search url for
Retrieving users
Translating user data
3400/3623
Creating search url for
Retrieving users


Retrieving users
Translating user data
3500/3623
Creating search url for
Retrieving users
Translating user data
3501/3623
Creating search url for
Retrieving users
Translating user data
3502/3623
Creating search url for
Retrieving users
Translating user data
3503/3623
Creating search url for
Retrieving users
Translating user data
3504/3623
Creating search url for
Retrieving users
Translating user data
3505/3623
Creating search url for
Retrieving users
Translating user data
3506/3623
Creating search url for
Retrieving users
Translating user data
3507/3623
Creating search url for
Retrieving users
Translating user data
3508/3623
Creating search url for
Retrieving users
Translating user data
3509/3623
Creating search url for
Retrieving users
Translating user data
3510/3623
Creating search url for
Retrieving users
Translating user data
3511/3623
Creating search url for
Retrieving users
Translating user data
3512/3623
Creating search url for
Retrieving users
Translating user data
3513/3623
Cr

Translating user data
3612/3623
Creating search url for
Retrieving users
Translating user data
3613/3623
Creating search url for
Retrieving users
Translating user data
3614/3623
Creating search url for
Retrieving users
Translating user data
3615/3623
Creating search url for
Retrieving users
Translating user data
3616/3623
Creating search url for
Retrieving users
Translating user data
3617/3623
Creating search url for
Retrieving users
Translating user data
3618/3623
Creating search url for
Retrieving users
Translating user data
3619/3623
Creating search url for
Retrieving users
Translating user data
3620/3623
Creating search url for
Retrieving users
Translating user data
3621/3623
Creating search url for
Retrieving users
Translating user data
3622/3623
Creating search url for
Retrieving users
Translating user data
3623/3623
Creating search url for
Retrieving users
Translating user data


In [24]:
# Export users dataframe
df_user.to_csv('users_'+start_date+'.csv')